In [1]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [3]:
# Python SQL toolkit and Object Relational Mapper
# import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func

In [4]:
from sqlalchemy import create_engine

In [11]:
import datetime as dt

In [5]:
# from Candidates import candidates_list

In [6]:
engine = create_engine("sqlite:///../db/twitter_db.sqlite")
conn = engine.connect()

In [7]:
tweets_df = pd.read_sql("SELECT * FROM tweet_data", conn)

In [8]:
tweets_df.head()

,id,created_at,created_at_time,created_at_date,created_at_datetime,tweet_id,tweet_id_str,full_text,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user_id,user_id_str,user_name,user_screen_name,retweet_count,favorite_count
0,1,Wed Jul 31 22:21:23 +0000 2019,22:21:23.000000,2019-07-31,2019-07-31 22:21:23.000000,1156691352983412737,1156691352983412737,"Here's the deal, President Trump inherited the...",NaN,None,NaN,None,None,939091,939091,Joe Biden,JoeBiden,938,4134
1,2,Wed Jul 31 21:28:00 +0000 2019,21:28:00.000000,2019-07-31,2019-07-31 21:28:00.000000,1156677917469896704,1156677917469896704,I’ve got some exciting news: I am now on Snapc...,NaN,None,NaN,None,None,939091,939091,Joe Biden,JoeBiden,173,790
2,3,Wed Jul 31 19:25:00 +0000 2019,19:25:00.000000,2019-07-31,2019-07-31 19:25:00.000000,1156646963506368512,1156646963506368512,A car accident took the lives of my first wife...,NaN,None,NaN,None,None,939091,939091,Joe Biden,JoeBiden,712,4376
3,4,Wed Jul 31 01:22:00 +0000 2019,01:22:00.000000,2019-07-31,2019-07-31 01:22:00.000000,1156374417691369474,1156374417691369474,"On what would have been his 46th birthday, my ...",NaN,None,NaN,None,None,939091,939091,Joe Biden,JoeBiden,210,1155
4,5,Tue Jul 30 22:12:23 +0000 2019,22:12:23.000000,2019-07-30,2019-07-30 22:12:23.000000,1156326700759863296,1156326700759863296,President Trump &amp; Betsy DeVos continue to ...,NaN,None,NaN,None,None,939091,939091,Joe Biden,JoeBiden,281,1215


In [12]:
tweets_df["created_at_datetime"] = tweets_df["created_at_datetime"].apply(lambda x: dt.datetime.strptime(x,
                                                                "%Y-%m-%d %H:%M:%S.%f"))

In [13]:
tweets_df["day"] = tweets_df["created_at_datetime"].apply(lambda x: dt.datetime.strftime(x, "%A"))
tweets_df["hour"] = tweets_df["created_at_datetime"].apply(lambda x: dt.datetime.strftime(x, "%H"))
tweets_df["month"] = tweets_df["created_at_datetime"].apply(lambda x: dt.datetime.strftime(x, "%B"))

In [14]:
tweets_df.drop(columns = ["created_at", "created_at_time", "created_at_date","created_at_datetime",
                    "tweet_id", "tweet_id_str", "in_reply_to_status_id",
                    "in_reply_to_status_id_str", "in_reply_to_user_id",
                    "in_reply_to_user_id_str", "in_reply_to_screen_name",
                    "user_id_str", "user_id", "user_screen_name", "id"], inplace = True)

In [15]:
tweets_df = tweets_df[["user_name", "month", "day", "hour", "retweet_count", "favorite_count", "full_text"]]

In [ ]:
tweets_df.head()

In [16]:
X_df = tweets_df[["full_text", "month", "day", "hour", "retweet_count", "favorite_count"]]
X_df = pd.get_dummies(X_df, columns = ["month", "day", "hour"])

In [17]:
X_df.head()

,full_text,retweet_count,favorite_count,month_April,month_August,month_December,month_February,month_January,month_July,month_June,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,"Here's the deal, President Trump inherited the...",938,4134,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,I’ve got some exciting news: I am now on Snapc...,173,790,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,A car accident took the lives of my first wife...,712,4376,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,"On what would have been his 46th birthday, my ...",210,1155,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,President Trump &amp; Betsy DeVos continue to ...,281,1215,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
import nltk
import re
import string
pd.set_option('display.max_colwidth', 100) # To extend column width

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [19]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

# NGramVectorizer
ngram_vect = CountVectorizer(ngram_range=(2,2), analyzer=clean_text)
X_count = ngram_vect.fit_transform(X_df['full_text'])
X_count_feat = pd.concat([X_df, pd.DataFrame(X_count.toarray(), columns=ngram_vect.get_feature_names())], axis=1)

In [21]:
X_count_feat.head()

,full_text,retweet_count,favorite_count,month_April,month_August,month_December,month_February,month_January,month_July,month_June,...,هستند,چهلسالشکست,که,۴۰,⅓,⅔,祝你新年快乐,광주민주화운동,𝐥𝐞𝐠𝐚𝐥,𝑽𝒂𝒎𝒐𝒏𝒐𝒔
0,"Here's the deal, President Trump inherited the economy from the Obama-Biden Administration. And ...",938,4134,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,I’ve got some exciting news: I am now on Snapchat! Subscribe to get a behind-the-scenes look at ...,173,790,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,A car accident took the lives of my first wife and baby daughter. My family was fortunate to hav...,712,4376,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,"On what would have been his 46th birthday, my thoughts are with the family of my friend Reverend...",210,1155,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,President Trump &amp; Betsy DeVos continue to roll back our progress on LGBTQ rights. No child s...,281,1215,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
X_count_feat = X_count_feat.loc[:, "retweet_count":"𝑽𝒂𝒎𝒐𝒏𝒐𝒔"]
# X_count_feat.drop(columns = ["full_text"], inplace = True)

MemoryError: 

In [ ]:
# X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)


In [ ]:
data_y = tweets_df.values
data_x = X.values
X = data_x[:, 0:]
y = data_y[:,0]

In [ ]:
X

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [ ]:
# for label, original_class in zip(encoded_y, y):
#     print('Original Class: ' + str(original_class))
#     print('Encoded Label: ' + str(label))
#     print('-' * 12)

In [ ]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
from sklearn.preprocessing import MinMaxScaler


In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
nltk.download('wordnet')

In [ ]:
import re
import string
pd.set_option('display.max_colwidth', 100) # To extend column width

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# count_vect = CountVectorizer(analyzer=clean_text)
# X_counts = count_vect.fit_transform(tweets_df['full_text'])
# print(X_counts.shape)
# print(count_vect.get_feature_names())

In [ ]:
# X_counts

In [ ]:
# X_counts_df = pd.DataFrame(X_counts.toarray(), columns=count_vect.get_feature_names())
# X_counts_df.head(10)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

ngram_vect = CountVectorizer(ngram_range=(2,2),analyzer=clean_text) # It applies only bigram vectorizer
X_counts = ngram_vect.fit_transform(tweets_df['full_text'])
print(X_counts.shape)
print(ngram_vect.get_feature_names())

In [ ]:
X_counts_df = pd.DataFrame(X_counts.toarray(), columns=ngram_vect.get_feature_names())
X_counts_df.head(10)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vect = TfidfVectorizer(analyzer=clean_text)
# X_tfidf = tfidf_vect.fit_transform(tweets_df['full_text'])
# print(X_tfidf.shape)
# print(tfidf_vect.get_feature_names())

In [ ]:
# X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names())
# X_tfidf_df.head(10)

In [ ]:
# print(X_tfidf_df.loc[(X_tfidf_df!=0).any(axis=1)])

In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
tweets_df.head()